In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import sklearn.cluster as c
import pickle

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/data/clean_data_wo_outliers.csv')
df['Nace2'] = df['Nace2'].astype(int)
df['gr_4digits'] = df['gr_4digits'].astype(int)

In [ ]:
df.head()

,Unnamed: 0,imones_kodas,imones_pavadinimas,imones_aprasymas,desc_len,Nace2,EmployeesLast,gr_stat_letter,gr_stat_code,gr_4digits
0,0,140891278,decopolis uab,kolektyvo padėti klientui susikurti jaukius ši...,47,477890,1.0,G,G47,4778
1,1,133852410,dakra uab,maketavimas spaudai internetui specialūs pasiū...,8,581900,2.0,J,J58,5819
2,3,304224333,mokymų erdvė uab,augti verslui asmenybei,3,855900,1.0,P,P85,8559
3,4,303425493,ausvitas uab,verslo konsultacijos struktūrinių paramos fond...,17,702200,0.0,M,M70,7022
4,5,304869540,dizaino kalvė mb,interjerų projektavimas įrengimas baldų projek...,12,741000,0.0,M,M74,7410


In [ ]:
print(f"Full Nace2 Code Unique Values: {df['Nace2'].nunique()}")
print(f"Grp letter Nace2 Code Unique Values: {df['gr_stat_letter'].nunique()}")
print(f"Grp. stat coded Nace2 Code Unique Values: {df['gr_stat_code'].nunique()}")
print(f"4 Digits Nace2 Code Unique Values: {df['gr_4digits'].nunique()}")

Full Nace2 Code Unique Values: 769
Grp letter Nace2 Code Unique Values: 20
Grp. stat coded Nace2 Code Unique Values: 82
4 Digits Nace2 Code Unique Values: 595


In [ ]:
n_clusters_to_pred = [20, 82, 595, 769]

# Loading datasets

In [ ]:
ae_bert = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/AE/bert_ae_embeddings.csv')
ae_labse = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/AE/labse_ae_embeddings.csv')
ae_w2v = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/AE/w2v_ae_embeddings.csv')

pca_bert = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/PCA/bert_pca_embeddings.csv')
pca_labse = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/PCA/labse_pca_embeddings.csv')
pca_w2v = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/PCA/w2v_pca_embeddings.csv')

svd_bert = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/SVD/bert_svd_embeddings.csv')
svd_labse = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/SVD/labse_svd_embeddings.csv')
svd_w2v = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/SVD/w2v_svd_embeddings.csv')

umap_bert = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/UMAP/bert_umap_embeddings.csv')
umap_labse = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/UMAP/labse_umap_embeddings.csv')
umap_w2v = pd.read_csv('/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/FeatureSelection/UMAP/w2v_umap_embeddings.csv')

datasets = [ae_bert, ae_labse, ae_w2v, pca_bert, pca_labse, pca_w2v,
            svd_bert, svd_labse, svd_w2v, umap_bert, umap_labse, umap_w2v]
dataset_names = ['AutoEncoder BERT', 'AutoEncoder LaBSE', 'AutoEncoder W2V',
                 'PCA BERT', 'PCA LaBSE', 'PCA W2V',
                 'SVD BERT', 'SVD LaBSE', 'SVD W2V',
                 'UMAP BERT', 'UMAP LaBSE', 'UMAP W2V']

In [ ]:
for dataset, dataset_name in zip(datasets, dataset_names):
  zeroes = dataset.columns[(dataset == 0).all()]
  if np.any(zeroes):
    print(f"{dataset_name} has zeros columns", end=' | ')
    print(f"Dropping zero columns", end=' | ')
    dataset.drop(zeroes, axis=1, inplace=True)
    print(f"Zero columns dropped")

AutoEncoder BERT has zeros columns | Dropping zero columns | Zero columns dropped
AutoEncoder LaBSE has zeros columns | Dropping zero columns | Zero columns dropped
AutoEncoder W2V has zeros columns | Dropping zero columns | Zero columns dropped


# Clustering

In [ ]:
PATH = '/content/drive/MyDrive/Univerui/0 MAGISTRAS/tiriamasis/clustering_outputs/'

## K-Means

In [ ]:
for _n_clusters in n_clusters_to_pred:
  print(f"Experiments with {_n_clusters}")
  results = pd.DataFrame()
  for dataset, name in zip(datasets, dataset_names):
    print(f"\tRunning {name} dataset")
    # Running clustering
    model = c.KMeans(n_clusters=_n_clusters, n_init='auto')
    preds = model.fit_predict(dataset.to_numpy())

    # Saving preds
    n = name.lower().replace(' ', '_')
    results[n] = preds

    # Saving clustering model
    with open(f"{PATH}models/kmeans/{n}_{_n_clusters}.pkl", "wb") as f:
      pickle.dump(model, f)
  results.to_csv(PATH+'kmeans_'+str(_n_clusters)+'.csv', index=False)


Experiments with 20
	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset
Experiments with 82
	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset
Experiments with 595
	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Ru

## DBSCAN (Better to use OPTIC, it's a successor and does not require to find epsilon)

https://www.analyticsvidhya.com/blog/2020/09/how-dbscan-clustering-works/?utm_source=reading_list&utm_medium=https://www.analyticsvidhya.com/blog/2016/11/an-introduction-to-clustering-and-different-methods-of-clustering/


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
results = {}
for dataset, name in zip(datasets, dataset_names):
  print(f"\tRunning {name} dataset")
  model = c.AffinityPropagation().fit(dataset.to_numpy())

  n = name.lower().replace(' ', '_')
  results[n] = {
      'preds': model.labels_,
      'centers': model.cluster_centers_indices_
  }

  with open(f"{PATH}models/affinity_propogation/{n}.pkl", "wb") as f:
      pickle.dump(model, f)

  with open(PATH+'affinity_propogation_results.pkl', 'wb') as file_pi:
      pickle.dump(results, file_pi)

## HDBSCAN

In [ ]:
results = {}
for dataset, name in zip(datasets, dataset_names):
  print(f"\tRunning {name} dataset")
  model = c.HDBSCAN().fit(dataset.to_numpy())

  n = name.lower().replace(' ', '_')
  results[n] = model.labels_

  with open(f"{PATH}models/hdbscan/{n}.pkl", "wb") as f:
      pickle.dump(model, f)

  with open(PATH+'hdbscan_results.pkl', 'wb') as file_pi:
      pickle.dump(results, file_pi)

	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset


## GMM

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
for _n_clusters in n_clusters_to_pred:
  print(f"Experiments with {_n_clusters}")
  results = {}
  for dataset, name in zip(datasets, dataset_names):
    print(f"\tRunning {name} dataset")
    # Running clustering
    model = GaussianMixture(n_components=_n_clusters, random_state=0)

    # Saving preds
    n = name.lower().replace(' ', '_')
    results[n] = {
        'preds': model.fit_predict(dataset.to_numpy()),
        'probs': model.predict_proba(dataset.to_numpy()),
    }

    # Saving clustering model
    with open(f"{PATH}models/gmm/{n}_{_n_clusters}.pkl", "wb") as f:
      pickle.dump(model, f)
  with open(PATH+'gmm_'+str(_n_clusters)+'.pkl', 'wb') as file_pi:
      pickle.dump(results, file_pi)


Experiments with 20
	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset
Experiments with 82
	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset
Experiments with 595
	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Ru

## BIRCH

In [ ]:
results = {}
for dataset, name in zip(datasets, dataset_names):
  print(f"\tRunning {name} dataset")
  model = c.Birch(branching_factor = 50, n_clusters = None, threshold = 1.5).fit(dataset.to_numpy())

  n = name.lower().replace(' ', '_')
  results[n] = model.predict(dataset.to_numpy())

  with open(f"{PATH}models/birch/{n}.pkl", "wb") as f:
      pickle.dump(model, f)

  with open(PATH+'birch_results.pkl', 'wb') as file_pi:
      pickle.dump(results, file_pi)

	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset


## Affinity Propogation (NOT USED, REQUIRES TOO MUCH RAM)

In [ ]:
results = {}
for dataset, name in zip(datasets, dataset_names):
  print(f"\tRunning {name} dataset")
  model = c.AffinityPropagation().fit(dataset.to_numpy())

  n = name.lower().replace(' ', '_')
  results[n] = {
      'preds': model.labels_,
      'centers': model.cluster_centers_indices_
  }

  with open(f"{PATH}models/affinity_propogation/{n}.pkl", "wb") as f:
      pickle.dump(model, f)

  with open(PATH+'affinity_propogation_results.pkl', 'wb') as file_pi:
      pickle.dump(results, file_pi)

	Running AutoEncoder BERT dataset


## Mean Shift

In [ ]:
results = {}
for dataset, name in zip(datasets, dataset_names):
  print(f"\tRunning {name} dataset")
  model = c.MeanShift().fit(dataset.to_numpy())

  n = name.lower().replace(' ', '_')
  results[n] = {
      'preds': model.labels_,
      'centers': model.cluster_centers_
  }

  with open(f"{PATH}models/mean_shift/{n}.pkl", "wb") as f:
      pickle.dump(model, f)

  with open(PATH+'mean_shift_results.pkl', 'wb') as file_pi:
      pickle.dump(results, file_pi)

	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset


In [ ]:
with open(PATH+'mean_shift_results.pkl', 'rb') as f:
    results = pickle.load(f)

In [ ]:
# ReRun after runtime disconected on last dataset training
model = c.MeanShift().fit(datasets[-1].to_numpy())

In [ ]:
n = dataset_names[-1].lower().replace(' ', '_')
results[n] = {
    'preds': model.labels_,
    'centers': model.cluster_centers_
}

with open(f"{PATH}models/mean_shift/{n}.pkl", "wb") as f:
    pickle.dump(model, f)

with open(PATH+'mean_shift_results_2.pkl', 'wb') as file_pi:
    pickle.dump(results, file_pi)

## OPTICS

In [ ]:
results = {}
for dataset, name in zip(datasets, dataset_names):
  print(f"\tRunning {name} dataset")
  model = c.OPTICS().fit(dataset.to_numpy())

  n = name.lower().replace(' ', '_')
  results[n] = model.labels_

  with open(f"{PATH}models/optics/{n}.pkl", "wb") as f:
      pickle.dump(model, f)

  with open(PATH+'optics_results.pkl', 'wb') as file_pi:
      pickle.dump(results, file_pi)

	Running AutoEncoder BERT dataset


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_optics.py:995: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


	Running AutoEncoder LaBSE dataset


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_optics.py:995: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


	Running AutoEncoder W2V dataset


...

	Running SVD LaBSE dataset


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_optics.py:995: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


	Running SVD W2V dataset


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_optics.py:995: RuntimeWarning: divide by zero encountered in divide
  ratio = reachability_plot[:-1] / reachability_plot[1:]


	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset


## Agglomerative Clustering

In [ ]:
for _n_clusters in n_clusters_to_pred:
  print(f"Experiments with {_n_clusters}")
  results = pd.DataFrame()
  for dataset, name in zip(datasets, dataset_names):
    print(f"\tRunning {name} dataset")
    # Running clustering
    model = c.AgglomerativeClustering(n_clusters=_n_clusters)
    preds = model.fit_predict(dataset.to_numpy())

    # Saving preds
    n = name.lower().replace(' ', '_')
    results[n] = preds

    # Saving clustering model
    with open(f"{PATH}models/agglomerative_clustering/{n}_{_n_clusters}.pkl", "wb") as f:
      pickle.dump(model, f)
  results.to_csv(PATH+'agglomerativeClustering_'+str(_n_clusters)+'.csv', index=False)


Experiments with 20
	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset
Experiments with 82
	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Running UMAP BERT dataset
	Running UMAP LaBSE dataset
	Running UMAP W2V dataset
Experiments with 595
	Running AutoEncoder BERT dataset
	Running AutoEncoder LaBSE dataset
	Running AutoEncoder W2V dataset
	Running PCA BERT dataset
	Running PCA LaBSE dataset
	Running PCA W2V dataset
	Running SVD BERT dataset
	Running SVD LaBSE dataset
	Running SVD W2V dataset
	Ru

## Spectral clustering (RAM ISSUES)

In [ ]:
for _n_clusters in n_clusters_to_pred:
  print(f"Experiments with {_n_clusters}")
  results = pd.DataFrame()
  for dataset, name in zip(datasets, dataset_names):
    print(f"\tRunning {name} dataset")
    # Running clustering
    model = c.SpectralClustering(n_clusters=_n_clusters,assign_labels='kmeans',random_state=0)
    preds = model.fit_predict(dataset.to_numpy())

    # Saving preds
    n = name.lower().replace(' ', '_')
    results[n] = preds

    # Saving clustering model
    with open(f"{PATH}models/spectral_clustering/{n}_{_n_clusters}.pkl", "wb") as f:
      pickle.dump(model, f)
  results.to_csv(PATH+'spectral_clustering_'+str(_n_clusters)+'.csv', index=False)


Experiments with 20
	Running AutoEncoder BERT dataset
